In [1]:
import data
import models
import soundfile as sf
import torch

import numpy as np
import pandas as pd
import subprocess
import shlex

In [2]:
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
config = data.read_config("experiments/no_unfreezing.cfg")
_, _, _ = data.get_SLU_datasets(config)
model = models.Model(config).eval()
model.load_state_dict(torch.load("experiments/no_unfreezing/training/model_state.pth", map_location=device)) # load trained model

# signal, _ = sf.read("audio1520623611766_1_1.wav")
# signal = torch.tensor(signal, device=device).float().unsqueeze(0)
#
# model.decode_intents(signal)
# phonemes = model.decode_phonemes(signal)
# intents = model.decode_intents(signal)
# print(intents)
# print(phonemes.size())
# print(phonemes)
# print(phonemes)
# print(torch.sum(phonemes, dim=0).size())
# print(torch.sum(phonemes, dim=0))
# print(torch.sum(phonemes, dim=1).size())
# print(torch.sum(phonemes, dim=1))
# values, indices = torch.max(phonemes, dim=1)
# print(indices.size())
# print(indices)

# print(model.pretrained_model.compute_posteriors(signal)[0].max(2)[1])

# with open("experiments/no_unfreezing/pretraining/phonemes_bkp_ipa.txt", "r") as f:
#     phonemes = f.readlines()
#     phonemes = [p.strip() for p in phonemes]
#     indices = model.pretrained_model.compute_posteriors(signal)[0].max(2)[1]
#     print(indices)
#     ipa_list = [phonemes[index.item()] for index in indices[0]]
#     print(ipa_list)
#     print(''.join(ipa_list))


lang = "Tamil"
# lang = "Sinhala"

base_path = '/home/isham/Projects/MSc/Speech_Intent/'

# audio_folder = 'data/formatted_data/'
# # read csv from csv
# data = pd.read_csv(base_path + 'data/formatted_data_v2.csv')
# file_names = data['filename']
# file_names = file_names.apply(lambda x: base_path + audio_folder + str(x))

audio_folder = 'data/' + lang + '_Dataset/audio_files/'
# read csv from csv
data = pd.read_csv(base_path + 'data/' + lang + '_Dataset/' + lang + '_Data.csv')
file_names = data['audio_file']
file_names = file_names.apply(lambda x: base_path + audio_folder + str(x))

file_names = file_names.values
ds2_features = []


maxShape = (0,42)
for f in tqdm(file_names):
    c_max_rows = 555
    signal, _ = sf.read(f)
    signal = torch.tensor(signal, device=device).float().unsqueeze(0)

    if len(signal.size()) > 2:
        # print('Stereo')
        command = 'ffmpeg -i ' + f + \
                  ' -ac 1 -ab 256000 -ar 16000 ' + f + ' -y'
        subprocess.run(shlex.split(command), stdout=subprocess.PIPE)

        signal, _ = sf.read(f)
        signal = torch.tensor(signal, device=device).float().unsqueeze(0)

    print('shape: ', signal.size(), ' len: ', len(signal.size()))

    phonemes = model.decode_phonemes(signal)
    out = phonemes.detach().numpy()
    maxShape = max(out.shape, maxShape)
    print(out.shape)
    print(out)
    out = np.pad(out, ((0, c_max_rows - out.shape[0]), (0, 0)), 'constant')
    print(out.shape)
    print(out)
    ds2_features.append(out)
    # ds2_features.append(phonemes.detach().numpy())
    # print((phonemes.detach().numpy()).shape)
    # print(out.shape)

ds2_features = np.array(ds2_features)
# print(ds2_features.shape)
# print(ds2_features)


np.save(base_path + 'data/' + lang + '_Dataset/phoneme_decode_' + lang + '_data_v2', ds2_features)

  0%|          | 1/400 [00:00<00:40,  9.97it/s]

shape:  torch.Size([1, 54144])  len:  2
(85, 42)
[[1.2989105e-01 5.1873787e-03 9.4881037e-04 ... 5.5831566e-05
  9.8997698e-06 9.9462015e-04]
 [9.7907089e-02 6.3742022e-04 1.2222477e-04 ... 9.5960977e-06
  1.3014256e-06 8.6418433e-05]
 [7.8293681e-02 2.7292315e-04 4.2752945e-05 ... 3.5283392e-06
  4.2151095e-07 2.7876309e-05]
 ...
 [3.1567590e-05 2.0879104e-04 1.2245722e-04 ... 1.0126039e-06
  2.3824987e-07 4.9554776e-05]
 [4.5093817e-05 5.3257233e-04 4.1588076e-04 ... 3.0559627e-06
  5.4051765e-07 9.2471455e-05]
 [7.4915471e-05 5.6434283e-03 8.9114634e-03 ... 2.2222508e-05
  9.1012107e-06 2.2459484e-03]]
(555, 42)
[[1.2989105e-01 5.1873787e-03 9.4881037e-04 ... 5.5831566e-05
  9.8997698e-06 9.9462015e-04]
 [9.7907089e-02 6.3742022e-04 1.2222477e-04 ... 9.5960977e-06
  1.3014256e-06 8.6418433e-05]
 [7.8293681e-02 2.7292315e-04 4.2752945e-05 ... 3.5283392e-06
  4.2151095e-07 2.7876309e-05]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]


  1%|          | 4/400 [00:00<00:41,  9.47it/s]

(85, 42)
[[1.08314492e-01 5.96749224e-03 1.17967918e-03 ... 3.81512473e-05
  1.18846565e-05 9.23753541e-04]
 [7.37299845e-02 8.65827664e-04 1.71676103e-04 ... 9.25426048e-06
  1.89370496e-06 9.10928793e-05]
 [5.62610403e-02 3.34110897e-04 5.52553865e-05 ... 3.61088109e-06
  5.18479112e-07 2.67748565e-05]
 ...
 [2.20810805e-04 4.12965222e-04 1.92879859e-04 ... 1.88035915e-06
  5.77466494e-07 6.24734457e-05]
 [2.82938185e-04 8.94015480e-04 6.41417399e-04 ... 5.02178727e-06
  1.14361626e-06 1.06527768e-04]
 [3.23295302e-04 7.17849797e-03 1.36515200e-02 ... 3.04780770e-05
  1.36325116e-05 2.30012345e-03]]
(555, 42)
[[1.08314492e-01 5.96749224e-03 1.17967918e-03 ... 3.81512473e-05
  1.18846565e-05 9.23753541e-04]
 [7.37299845e-02 8.65827664e-04 1.71676103e-04 ... 9.25426048e-06
  1.89370496e-06 9.10928793e-05]
 [5.62610403e-02 3.34110897e-04 5.52553865e-05 ... 3.61088109e-06
  5.18479112e-07 2.67748565e-05]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e

  2%|▏         | 6/400 [00:00<00:41,  9.51it/s]

...

(555, 42)
[[1.6823798e-01 4.5387191e-03 7.4508309e-04 ... 3.2391010e-05
  7.0480874e-06 5.9648184e-04]
 [1.4947060e-01 5.8343593e-04 6.3830856e-05 ... 6.2685408e-06
  7.4033778e-07 4.4757497e-05]
 [1.3252392e-01 2.7262396e-04 2.3912820e-05 ... 2.6550322e-06
  2.1241684e-07 1.6449985e-05]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
shape:  torch.Size([1, 48768])  len:  2
(77, 42)
[[1.4390196e-01 5.2209417e-03 2.4353424e-03 ... 9.6147029e-05
  1.1989284e-05 4.5063272e-03]
 [5.4734994e-02 2.3721450e-03 6.8882725e-04 ... 5.6256886e-05
  2.3858026e-06 5.3484468e-03]
 [4.3170191e-02 1.8258005e-03 4.1996394e-04 ... 4.5685902e-05
  1.5605387e-06 3.6467495e-03]
 ...
 [8.2933700e-05 8.3410932e-04 9.7265973e-04 ... 7.8906833e-06
  1.2519213e-06 1.1215950e-03]


100%|█████████▉| 398/400 [00:44<00:00, 11.55it/s]

(61, 42)
[[3.2298613e-01 7.6794433e-03 1.1040368e-03 ... 4.0455572e-05
  6.4622882e-06 8.7198557e-04]
 [3.2913226e-01 2.6280659e-03 1.8042864e-04 ... 1.2106170e-05
  1.2191530e-06 1.0660154e-04]
 [3.1759211e-01 6.3554291e-03 1.1823908e-04 ... 9.6623417e-06
  6.9771568e-07 1.1809236e-04]
 ...
 [5.5914308e-04 4.7377506e-04 8.1387509e-05 ... 2.4756432e-06
  7.4011427e-07 1.6121659e-04]
 [5.4394698e-04 5.3867372e-04 7.4062307e-05 ... 2.6089554e-06
  1.2104327e-06 1.4162490e-04]
 [4.6840796e-04 1.8129407e-03 9.8782673e-04 ... 1.7677827e-05
  7.1184081e-06 6.8843405e-04]]
(555, 42)
[[3.2298613e-01 7.6794433e-03 1.1040368e-03 ... 4.0455572e-05
  6.4622882e-06 8.7198557e-04]
 [3.2913226e-01 2.6280659e-03 1.8042864e-04 ... 1.2106170e-05
  1.2191530e-06 1.0660154e-04]
 [3.1759211e-01 6.3554291e-03 1.1823908e-04 ... 9.6623417e-06
  6.9771568e-07 1.1809236e-04]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e

100%|██████████| 400/400 [00:44<00:00, 10.67it/s]

(154, 42)
[[9.3195431e-02 4.5642708e-03 9.9617755e-04 ... 4.9137776e-05
  7.5601547e-06 1.1007563e-03]
 [5.9898823e-02 7.1586546e-04 1.1952290e-04 ... 8.2739743e-06
  1.1551867e-06 9.9317484e-05]
 [3.9304178e-02 3.0556470e-04 4.0152448e-05 ... 2.9533846e-06
  3.4889609e-07 3.2594406e-05]
 ...
 [1.6189279e-04 2.4832596e-04 1.1835614e-04 ... 1.0823373e-06
  2.7460646e-07 5.0116287e-05]
 [2.0595065e-04 4.8997178e-04 3.5240289e-04 ... 2.5755521e-06
  5.7040802e-07 8.4602623e-05]
 [2.3613955e-04 4.4119875e-03 7.9896562e-03 ... 1.9043986e-05
  8.2201395e-06 1.8753889e-03]]
(555, 42)
[[9.3195431e-02 4.5642708e-03 9.9617755e-04 ... 4.9137776e-05
  7.5601547e-06 1.1007563e-03]
 [5.9898823e-02 7.1586546e-04 1.1952290e-04 ... 8.2739743e-06
  1.1551867e-06 9.9317484e-05]
 [3.9304178e-02 3.0556470e-04 4.0152448e-05 ... 2.9533846e-06
  3.4889609e-07 3.2594406e-05]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000

In [3]:
out[100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [4]:
maxShape

(256, 42)